In [1]:
ls

deerwester.dict        hey.csv                     new_ddoc2.txt
deerwester2.dict       my_town.ipynb               new_doc2.txt
deerwester2.mm         my_town2-Copy3.ipynb        prepre.py
deerwester3.dict       my_town3-Copy1.ipynb        prepre.pyc
deerwester3.mm         my_town3_todai-Copy1.ipynb  qqq1.csv
deerwester_todai.dict  my_town3_todai-Copy2.ipynb  qqq2_pesdo.csv
deerwester_todai.mm    my_town3_todai.ipynb        result.png
deerwesterr.mm         my_towwn.ipynb              test.txt
doc2_vec.ipynb         new_answer.csv              todai_pesdo.csv
experi.txt             new_ccc.csv                 untitled.txt


In [1]:
__author__ = 'masatoshi'

import csv
import numpy as np
a=[]

with open('hey.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        a.append(row)

####print len(a)

b=[]
for kkk in range(len(a)):
    if(len(a[kkk])==1):
        b.append(a[kkk][0])

###print b[0]

from gensim import corpora, models, similarities

documents = ["Human machine interface","a survey of user opinion","graph minors a survery"]

documents = b

stoplist = set('for a of the and to in'.split())

texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

###print texts

dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=2, no_above=0.3)
dictionary.save('/tmp/deerwester.dict')
corpus = [dictionary.doc2bow(text) for text in texts]

###print corpus

tfidf = models.TfidfModel(corpus)

corpus_tfidf = tfidf[corpus]
answer =[]
for doc in corpus_tfidf:
    ###print(doc)
    answer.append(doc)

kkkk = len(dictionary.token2id)
###print len(answer)
c=[]
###print answer[0]
for i in answer:
    p = np.zeros(kkkk)
    for j in i:
        p[j[0]] = j[1]
    c.append(p)

print np.shape(c)
print 
print len(answer[1])
###print(dictionary.token2id)

train_X = c

import matplotlib

(486, 5699)

99


Using gpu device 0: GRID K520


In [71]:
print train_X[0]

[ 0.  0.  0. ...,  0.  0.  0.]


In [ ]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi[corpus_tfidf]
hey1 =[]
hey2 =[]
for i in corpus_lsi:
    hey1.append(i[0][1])
    hey2.append(i[1][1])                 

print hey[0:10]
plt.scatter(hey1,hey2)
plt.show()

[[0.21505615184552229, 0.052921379934237879], [0.20422945212479837, 0.027914987518129358], [0.25892551893546345, 0.072101435235729816], [0.20573119563240394, 0.06542110067731477], [0.22151311922451283, -0.019670088069680923], [0.21208524952866081, -0.015566306658592655], [0.24084929257096949, 0.12492761361910854], [0.18457733671168916, 0.10477193185162932], [0.1818774700820367, 0.0020684408040738964], [0.18962196432175565, 0.082134569797680163]]


In [2]:
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
from collections import OrderedDict
rng = np.random.RandomState(1234)

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata

###print mnist_x.shape

In [20]:
#Autoencoder (or RBM)
class Autoencoder:
    def __init__(self,visible_dim,hidden_dim,W,function):
        ## WRITE ME
        self.visible_dim = visible_dim
        self.hidden_dim = hidden_dim
        self.function = function
        self.W =W
        self.a = theano.shared(np.zeros(visible_dim).astype(np.float32),name="a")
        self.b = theano.shared(np.zeros(hidden_dim).astype(np.float32),name="b")
        self.params = [self.W,self.a,self.b]

    #encoder
    def encode(self,x):
        ## WRITE ME
        u = T.dot(x, self.W)+self.b
        y = self.function(u)
        return y
        
    #decoder
    def decode(self,x):
        ## WRITE ME
        u = T.dot(x, self.W.T)+self.a
        y = self.function(u)
        return y
    
    #forward propagation
    def prop(self,x):
        ## WRITE ME
        y = self.encode(x)
        reconst_x = self.decode(y)
        return reconst_x
    
    #reconstruction error
    def reconst_error(self,x,noise):
        tilde_x = x*noise
        reconst_x = self.prop(tilde_x)
        error = T.mean(T.sum(T.nnet.binary_crossentropy(reconst_x,x),axis=1))
        return error, reconst_x
    
#SGD
def sgd(params,gparams,lr=0.1):
    updates = OrderedDict()
    for param, gparam in zip(params, gparams):
        updates[param] = param - lr * gparam
    return updates
  

#Multi Layer Perceptron
class Layer:
    def __init__(self, in_dim, out_dim, function):
        self.W = theano.shared(rng.uniform(low=-np.sqrt(6. / (in_dim + out_dim)),high =np.sqrt(6. / (in_dim + out_dim)),size =(in_dim,out_dim)).astype('float32'), name = 'W')
        self.b = theano.shared(np.zeros(out_dim).astype('float32'), name = 'bias')
        self.params = [ self.W, self.b]
        
        self.W1 = theano.shared(0*rng.uniform(low=-np.sqrt(6. / (in_dim + out_dim)),high =np.sqrt(6. / (in_dim + out_dim)),size =(in_dim,out_dim)).astype('float32'), name = 'W')
        self.b1 = theano.shared(np.zeros(out_dim).astype('float32'), name = 'bias')
        self.momentums = [ self.W1, self.b1]       
             
        self.function = function
        self.h = None
        self.in_dim = in_dim
        self.out_dim =out_dim

    def fprop(self, x):
        h=self.function(theano.tensor.dot(x,self.W)+self.b)
        self.h = h
        return h
    
    def set_pretraining(self):
        ae = Autoencoder(self.in_dim,self.out_dim,self.W,self.function)
        ###noise = T.matrix('noise')
        ###noise =rng.binomial(size=train_X[start:end].shape, n=1, p=1-corruption_level)
        ##start = i*batch_size
        ##end = (i+1)*batch_size
        self.params[0] =ae.params[0]
        x  = T.matrix('X')
        noise = T.matrix('noise')
        ###noise =rng.binomial(size=x.shape, n=1, p=1-corruption_level)
        cost,reconst_x = ae.reconst_error(x,noise)
        params  = ae.params
        gparams = T.grad(cost, params)
        updates = sgd(params,gparams)
        self.pretraining = theano.function([x,noise], [cost,reconst_x], updates=updates, allow_input_downcast=True)
        hidden = ae.encode(x)
        self.encode_function = theano.function([x], hidden, allow_input_downcast=True)
#Pre-training
X = np.array(train_X)
print X.shape

def fff(x):
    return x

layers = [Layer(X.shape[1],2,T.nnet.sigmoid)]

def sgd(params,gparams,lr=0.1):
    updates = OrderedDict()
    for param, gparam in zip(params, gparams):
        updates[param] = param - lr * gparam
    return updates



(486, 5699)


In [21]:
for l, layer in enumerate(layers):
    corruption_level = 0.02## WRITE ME
    batch_size = 5
    X = shuffle(X)
    nbatches = X.shape[0] // batch_size
    ###print X.shape
    ###print nbatches       
    layer.set_pretraining()
    for epoch in range(20):
        err_all = []
        reconst_all =[]
        for i in range(nbatches):
            start = i*batch_size
            end = (i+1)*batch_size
            noise =rng.binomial(size=X[start:end].shape, n=1, p=1-corruption_level)
            err,recosnt = layer.pretraining(X[start:end],noise)
            err_all.append(err)
            reconst_all.append(recosnt)
        print reconst_all[0].shape
        print "Pre-training:: layer:%d, Epoch:%d, Error:%lf" %(l,epoch, np.mean(err_all))
    X = layer.encode_function(X)  

(5, 5699)
Pre-training:: layer:0, Epoch:0, Error:795.136108
(5, 5699)
Pre-training:: layer:0, Epoch:1, Error:176.416855
(5, 5699)
Pre-training:: layer:0, Epoch:2, Error:118.711349
(5, 5699)
Pre-training:: layer:0, Epoch:3, Error:96.387024
(5, 5699)
Pre-training:: layer:0, Epoch:4, Error:84.717323
(5, 5699)
Pre-training:: layer:0, Epoch:5, Error:77.629997
(5, 5699)
Pre-training:: layer:0, Epoch:6, Error:72.921722
(5, 5699)
Pre-training:: layer:0, Epoch:7, Error:69.596863
(5, 5699)
Pre-training:: layer:0, Epoch:8, Error:67.150322
(5, 5699)
Pre-training:: layer:0, Epoch:9, Error:65.278595
(5, 5699)
Pre-training:: layer:0, Epoch:10, Error:63.814487
(5, 5699)
Pre-training:: layer:0, Epoch:11, Error:62.642971
(5, 5699)
Pre-training:: layer:0, Epoch:12, Error:61.686920
(5, 5699)
Pre-training:: layer:0, Epoch:13, Error:60.899193
(5, 5699)
Pre-training:: layer:0, Epoch:14, Error:60.238403
(5, 5699)
Pre-training:: layer:0, Epoch:15, Error:59.679440
(5, 5699)
Pre-training:: layer:0, Epoch:16, Err

In [22]:
#train_size = len(train_data)
print reconst_all[0].shape
print X.shape
print X[0]
print X[1:10]

(5, 5699)
(486, 2)
[ 0.9980008   0.99800867]
[[ 0.99999046  0.99999058]
 [ 1.          0.99999964]
 [ 0.99876392  0.99876744]
 [ 0.99908757  0.99907684]
 [ 0.9997772   0.99977058]
 [ 0.99942654  0.9994536 ]
 [ 1.          1.        ]
 [ 0.99998713  0.99998701]
 [ 0.9987852   0.99878186]]


In [ ]:
#train_X, train_y = encode_dataset(train_data, word2index, tag2index)
#test_X  , test_y   = encode_dataset(dev_data,   word2index, tag2index)



from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

rng = numpy.random.RandomState(42)
trng = RandomStreams(42)


def sharedX(X, dtype="float32"):
    return theano.shared(numpy.asarray(X, dtype=dtype))


class Activation:
    def __init__(self, func):
        self.func = func
        self.params = []

    def fprop(self, x):
        return self.func(x)


class Projection:
    def __init__(self, in_dim, out_dim, scale=0.5):
        self.W = sharedX(rng.randn(in_dim, out_dim) * scale)
        self.params = [self.W]

    def fprop(self, x):
        h = self.W[x]
        return h
    
    
class Linear:
    def __init__(self, in_dim, out_dim, func,scale=0.5):
        self.W = sharedX(rng.randn(in_dim, out_dim) * scale)
        self.b = sharedX(rng.randn(out_dim,) * scale)
        self.h = None
        self.params = [ self.W, self.b ]
        self.func = func
    def fprop(self, x):
        c = T.dot(x, self.W)+self.b
        h = self.func(c)
        self.h = h
        return h



class RNN:
    def __init__(self, in_dim, hid_dim, func,scale=0.05):
        self.scale = scale
        self.hid_dim = hid_dim
        self.func = func
        ## 重みの次元を決める。
        self.Wx = sharedX(rng.randn(hid_dim,hid_dim ) * scale)
        self.Wh = sharedX(rng.randn(in_dim,hid_dim) * scale)
        self.bh = sharedX(rng.randn(hid_dim) * scale)
        ###self.Wy = sharedX(rng.randn(hid_dim,out_dim ) * scale)
        ###self.by = sharedX(rng.randn(out_dim ) * scale)
        
        ## Initial State をどのように初期化するか
        self.h0 = sharedX(rng.randn(hid_dim) * scale)
        self.output_info = [ self.h0 ]
        self.params = [self.Wh,self.bh,self.Wx]
    
    def fprop(self, x):
        def step(u_t, h_tm1):
            h = self.func(T.dot(h_tm1,self.Wx)+T.dot(u_t,self.Wh)+self.bh)
            ###self.output_info.append(h)
            return h
        ## Scan の方法を考える 
        h, _ = theano.scan(fn=step,sequences=x, outputs_info=self.h0)
        return h
    

def sgd(cost, params, lr):
    gparams = T.grad(cost, params)
    updates = OrderedDict()
    for param, gparam in zip(params, gparams):
        ## Advanced Gradient Glip を実装する　（必須ではない）
        #WRITE ME
        updates[param] = param - lr * gparam
    return updates

def prop(layers, x):
    for i, layer in enumerate(layers):
        if i == 0:
            layer_out = layer.fprop(x)
        else:
            layer_out = layer.fprop(layer_out)
    return layer_out


def get_params(layers):
    params = []
    for layer in layers:
        params += layer.params
    return params


### build Model + Train
vocab_size = len(word2index)
print vocab_size
hid_dim    = 150
out_dim    = len(tag2index)
in_dim = 400
x, t = T.lvector("x"), T.lvector("t")        
layers =[Projection(vocab_size,in_dim),RNN(in_dim,hid_dim,T.tanh),Linear(hid_dim,out_dim,T.nnet.softmax)]


train_X_pesdo =[i[::-1] for i in train_X]
train_y_pesdo =[i[::-1] for i in train_y]

                                     
###layer =[Projection(in_dim,vocab_size),]
prob = prop(layers, x) 

cost = - T.mean((T.log(prob))[T.arange(x.shape[0]), t])# Loss function を決める　 prop
pred =  T.argmax(prob, axis=1)
##予測した確率から、予測値を決める T.mean

## Collect Parameters
params = get_params(layers) 

## Define update graph
updates = sgd(cost, params, lr=numpy.float32(0.01)) 
updates2 = sgd(cost, params, lr=numpy.float32(0.001))
updates3 = sgd(cost, params, lr=numpy.float32(0.0001))
## Compile Function
train = theano.function([x,t], cost, updates=updates)
train2 = theano.function([x,t], cost, updates=updates2)
train3 = theano.function([x,t], cost, updates=updates3)
valid = theano.function([x,t], [cost, pred])
#test  = theano.function([x]　,　pred)

epochs = 8
## Train

train_X = train_X + train_X_pesdo
train_y = train_y + train_y_pesdo
for epoch in range(epochs):
    train_X, train_y = shuffle(train_X, train_y)  # Shuffle Samples !!
    for i, (instance_x, instance_y) in enumerate(zip(train_X, train_y)):
        if(epoch<=3):
            cost = train(instance_x, instance_y)
        elif(4<=epoch<=6):
            cost = train2(instance_x, instance_y)
        else:
            cost = train3(instance_x, instance_y)
        ###print cost
        if i % 1000 == 0:
            print "EPOCH:: %i, Iteration %i, cost: %.3f"%(epoch+1, i, cost)
    
    dev_true, pred_y = [], []
    dev_cost = []
    for i, (instance_x, instance_y) in enumerate(zip(test_X, test_y)):
        cost, pred = valid(instance_x, instance_y)
        pred_y += list(pred) # 予測結果はベクトル
        dev_true += instance_y
        dev_cost.append(cost)
    ###print dev_cost
    print classification_report(dev_true,pred_y)            
   
    